In [ ]:
"""
network_analysis_examples.ipynb

Created on Sun Mar 19 2023

@author: Lukas

This notebook serves as a playground for the code in the
network analysis repo.
"""

In [ ]:
# import packages

import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
# create an empty pandas dataframe with columns "Name", "Type", "Suppliers", and "Shareholders"

df = pd.DataFrame(columns=["Name", "Type", "Suppliers", "Shareholders"])


# populate the dataframe

df = df.append({"Name": "Company A", "Type": "Company", "Suppliers": ["Company B", "Company C"], "Shareholders": ["Bank A", "Individual A"]}, ignore_index=True)
df = df.append({"Name": "Company B", "Type": "Company", "Suppliers": ["Company C", "Company D"], "Shareholders": ["Bank A", "Individual B"]}, ignore_index=True)
df = df.append({"Name": "Company C", "Type": "Company", "Suppliers": ["Company D"], "Shareholders": ["Bank B", "Individual A"]}, ignore_index=True)
df = df.append({"Name": "Company D", "Type": "Company", "Suppliers": [], "Shareholders": ["Bank B", "Individual C"]}, ignore_index=True)

In [ ]:
# create a directed graph of suppliers from the dataframe

G = nx.DiGraph()

for index, row in df.iterrows():
    for supplier in row["Suppliers"]:
        G.add_edge(supplier, row["Name"])

# draw the graph, move the labels above the nodes so they don't overlap

pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True, node_size=500, node_color="orange")
labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels=labels, label_pos=0.3, font_size=8)
plt.show()